<a href="https://colab.research.google.com/github/AndySAnker/ClusterFinder/blob/main/ClusterFinder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ClusterFinder
**Github:** https://github.com/AndySAnker/ClusterFinder

**Paper:** ClusterFinder: XXXX

**Questions:** andy@chem.ku.dk 

Welcome to ClusterFinder that is a motif-extractor algorithm which can extract a cluster motif from a starting template and a pair distribution function (PDF).

This script guides you through a simple example of how to use ClusterFinder to find a cluster from a given PDF.

In [78]:
import numpy as np
import matplotlib as mpl
import time, random
from tqdm import tqdm
from scipy.optimize.minpack import leastsq
from diffpy.Structure import Structure, Atom
from diffpy.srfit.pdf import PDFContribution, PDFParser, PDFGenerator
from diffpy.srfit.fitbase import FitRecipe, FitResults, Profile, FitContribution
from diffpy.srreal.pdfcalculator import DebyePDFCalculator
from multiprocessing import Pool
from functools import partial
from ase.io import read

random.seed(14)
np.random.seed(14)

In [100]:
def structure_catalogue_maker(Number_of_atoms):
    """Makes a catalogue of structures"""
    
    structure_catalogue = np.ones((Number_of_atoms+1,Number_of_atoms))
    structure_catalogue[np.array([range(Number_of_atoms)]),np.array([range(Number_of_atoms)])] = 0
    return structure_catalogue

def Load(Experimental_Data, starting_model):
    """This function loads the data and structure"""
    
    # Get experimental data
    for skip_row in range(100):
        try:
            Exp_data = np.loadtxt(Experimental_Data, skiprows=skip_row)
        except ValueError:
            continue
    # Nyquist sampling the data by only using every 10nth datapoint
    if Exp_data[1,0] - Exp_data[0,0] < 0.1:
        Exp_r, Exp_Gr = Exp_data[::10,0], Exp_data[::10,1]
    else:
        Exp_r, Exp_Gr = Exp_data[:,0], Exp_data[:,1]
    # Normalise data
    Exp_Gr /= max(Exp_Gr)

    # Read structure and divide it into two lists: Atoms we want to iterate (W) and atoms we do not iterate (O)
    struct=[]
    with open(starting_model, 'r') as fi:
        for line in fi.readlines():
            sep_line=line.strip('{}\n\r ').split()
            if len(sep_line)==4: #  tillader andre informationer i xyz filen some ikke skal laeses
                struct.append(sep_line)
    elements=np.array(struct)[:,0]
    xyz=(np.array(struct)[:,1:].astype(float))
    
    return Exp_r, Exp_Gr, elements, xyz

def format_XYZ(starting_model, allowed_atoms):
	# Read structure and divide it into two lists: Atoms we want to iterate and atoms we do not iterate.
	# Save the file in this new format and get number of atoms that we iterate.
	permutable_struct = []
	nonpermutable_struct = []
	with open(starting_model, 'r') as fi:
		for line in fi.readlines():
			sep_line=line.strip('{}\n\r ').split()
			if len(sep_line)==4: #  tillader andre informationer i xyz filen some ikke skal laeses
				if sep_line[0] in allowed_atoms:
					permutable_struct.append(sep_line)
				else:
					nonpermutable_struct.append(sep_line)
	struct = permutable_struct + nonpermutable_struct

	New_XYZ = open(starting_model, "w")
	New_XYZ.write(str(len(struct)) + "\n\n")
	for i in range(len(struct)):
		New_XYZ.write(str(struct[i]).replace("[", "").replace("]", "").replace("'", "").replace(",", "") + "\n")
	New_XYZ.close()

	Num_permutable_atoms = len(permutable_struct)
	return Num_permutable_atoms

def create_cluster(structure_catalogue, xyz, atom_ph, index):
    """This function takes in a 'starting_model', and an 'index' from the 'structure_catalogue'. It generates the 
    corresponding structure."""

    xyz_Mo = xyz[:NumMo].copy()
    xyz_O = xyz[NumMo:len(xyz)].copy()
    keep_O = np.zeros(len(xyz_O))
    # Cycle through W atoms and delete W according to index 0's from permutation
    delete_M = np.where(np.array(structure_catalogue)[index,:] == 0)[0]

    # Delete atoms from starting model 
    xyz_Mo = np.delete(xyz_Mo, delete_M, 0)

    # Cycle through all atoms that is not iteratable and test if it is within the threshold distance. Delete atoms with no bonds
    for j in range(len(xyz_O)):
        dists = np.sqrt((xyz_Mo[:,0]-xyz_O[j,0])**2+(xyz_Mo[:,1]-xyz_O[j,1])**2+(xyz_Mo[:,2]-xyz_O[j,2])**2)
        if np.min(dists) < threshold:    
            keep_O[j] = 1

    # Cycle through W atoms and delete W according to index 0's from permutation
    delete_O = np.where(np.array(keep_O) == 0)[0]
    # Delete atoms from starting model 
    xyz_O = np.delete(xyz_O, delete_O, 0)

    # Create structure for iterable (W) and non-iterable (O) atoms and combine them
    Mo_cluster = Structure([Atom(atom_ph, xi) for xi in xyz_Mo])
    O_cluster = Structure([Atom('O', xi) for xi in xyz_O])
    cluster = Mo_cluster + O_cluster

    return cluster

def fitting(structure_catalogue, xyz, atom_ph, Qmin, Qmax, Qdamp, rmin, rmax, plot, index):
    """This function takes in a 'starting_model', and an 'index' from the 'structure_catalogue'. It generates the 
    corresponding structure and fit it to the 'Experimental_Data'."""

    # Create the cluster
    cluster = create_cluster(structure_catalogue, xyz, atom_ph, index)

    # Make a standard cluster refinement using Diffpy-CMI
    # Import the data and make it a PDFprofile. Define the range of the data that will be used in the fit.
    pdfprofile = Profile()
    pdfparser = PDFParser()
    pdfparser.parseFile(Experimental_Data)
    pdfprofile.loadParsedData(pdfparser)
    pdfprofile.setCalculationRange(xmin = rmin, xmax = rmax)

    # Setup the PDFgenerator that calculates the PDF from the structure
    pdfgenerator_cluster = PDFGenerator("G")
    # Add the profile and both generators to the PDFcontribution
    pdfcontribution = FitContribution("pdf")
    pdfcontribution.setProfile(pdfprofile, xname="r") 
    pdfcontribution.addProfileGenerator(pdfgenerator_cluster)
    
    pdfgenerator_cluster.setQmin(Qmin)
    pdfgenerator_cluster.setQmax(Qmax)
    pdfgenerator_cluster._calc.evaluatortype = 'OPTIMIZED'
    pdfgenerator_cluster.setStructure(cluster, periodic = False)

    # Use scaling factors proportional to molar content
    pdfcontribution.setEquation('mc*G')

    # Define the recipe to do the fit and add it to the PDFcontribution
    recipe = FitRecipe()
    recipe.addContribution(pdfcontribution)

    # Avoid too much output during fitting 
    recipe.clearFitHooks()

    # Add the scale factor.
    recipe.addVar(pdfcontribution.mc, 1.0, tag = "scale")
    
    # Add the instrumental parameters to the two generators
    pdfgenerator_cluster.qdamp.value = Qdamp
    #pdfgenerator_cluster.qbroad.value = 0.00

    # Add the delta2 parameters, and make sure it cannot take unphysical values
    #recipe.addVar(pdfgenerator_cluster.delta2, 0, name = "delta2_cluster", tag = "delta2")
    #recipe.restrain("delta2_cluster", lb=0, ub = 7, sig=0.001);
    
    # Add ADP and "cell" for the cluster
    phase_cluster = pdfgenerator_cluster.phase
    atoms = phase_cluster.getScatterers()
    lat = phase_cluster.getLattice()

    recipe.newVar("zoomscale", 1, tag = "lat")
    recipe.constrain(lat.a, 'zoomscale')
    recipe.constrain(lat.b, 'zoomscale')
    recipe.constrain(lat.c, 'zoomscale')
    recipe.restrain("zoomscale", lb=0.95, ub = 1.05, sig=0.001)

    Mo_cluster = recipe.newVar("Mo_Biso_cluster", 0.3, tag = 'adp_Mo')
    O_cluster = recipe.newVar("O_Biso_cluster", 0.4, tag = 'adp_O')

    for atom in atoms:
        if atom.element.title() == atom_ph:
            recipe.constrain(atom.Biso, Mo_cluster)
        elif atom.element.title() == "O":
            recipe.constrain(atom.Biso, O_cluster)
    recipe.restrain("Mo_Biso_cluster", lb=0.08, ub = 3, sig=0.001)
    recipe.restrain("O_Biso_cluster", lb=0.08, ub = 3, sig=0.001)

    pos_limit1 = 0.05
    pos_limit2 = 0.02
    pos_limit3 = 0.02
    for atom in atoms:
        if atom.element == atom_ph:
            recipe.addVar(atom.x, name=atom.name+'_x1', tag='xyz1')
            recipe.addVar(atom.y, name=atom.name+'_y1', tag='xyz1')
            recipe.addVar(atom.z, name=atom.name+'_z1', tag='xyz1')       
            #restrain atomic positions
            lbx = atom.x.value-pos_limit1
            ubx = atom.x.value+pos_limit1
            recipe.restrain(atom.x, lb=lbx, ub = ubx, sig=0.001)
            lby = atom.y.value-pos_limit1
            uby = atom.y.value+pos_limit1
            recipe.restrain(atom.y, lb=lby, ub = uby, sig=0.001)
            lbz = atom.z.value-pos_limit1
            ubz = atom.z.value+pos_limit1
            recipe.restrain(atom.y, lb=lby, ub = uby, sig=0.001)
    
    #free parameters are set
    recipe.fix('all')
    recipe.free("scale")
    #leastsq(recipe.residual, recipe.getValues())
    recipe.free("lat")
    leastsq(recipe.residual, recipe.getValues())
    #recipe.free("adp_Mo")
    #leastsq(recipe.residual, recipe.getValues())
    #recipe.free("adp_O")
    #leastsq(recipe.residual, recipe.getValues())
    #recipe.free("xyz1")
    #leastsq(recipe.residual, recipe.getValues())
    
    # Turn off printout of iteration number.
    #recipe.clearFitHooks()
    
    # We calculate the goodness-of-fit, Rwp
    g = recipe.pdf.profile.y
    gcalc = recipe.pdf.evaluate()
    rfactor1 = np.sqrt(sum((g - gcalc)**2) / sum((g)**2))
    
    # if plot == 1 it will also plot the fit
    if plot == 1:
        print ("FIT RESULTS\n")
        res1 = FitResults(recipe)
        print (res1)

        # Plot the observed and refined PDF.
        # Get the experimental data from the recipe
        r = recipe.pdf.profile.x
        gobs = recipe.pdf.profile.y

        # Get the calculated PDF and compute the difference between the calculated and measured PDF
        gcalc = recipe.pdf.evaluate()
        baseline = 1.1 * gobs.min()
        gdiff = gobs - gcalc

        # Plot!
        plt.figure()
        plt.plot(r, gobs, 'bo', label="G(r) data")
        plt.plot(r, gcalc, 'r-', label="G(r) fit")
        plt.plot(r, gdiff + baseline, 'g-', label="G(r) diff")
        plt.plot(r, np.zeros_like(r) + baseline, 'k:')
        plt.xlabel(r"$r (\AA)$")
        plt.ylabel(r"$G (\AA^{-2})$")
        plt.legend()

        plt.show()
    return 1-rfactor1

def fitting_multiprocess(structure_catalogue, xyz, atom_ph, Qmin, Qmax, Qdamp, rmin, rmax, SaveName, cores=1):
    """This function runs the refinement of all the structures in the structure catalogue using multiprocessing"""
    # Set up multiprocessing refinement
    fitindex = range(len(structure_catalogue))
    plot = 0
    
    func = partial(fitting, structure_catalogue, xyz, atom_ph, Qmin, Qmax, Qdamp, rmin, rmax, plot)
    
    # Use 'with' statement for pool
    with Pool(processes=cores) as p:
        # Wrap p.map with tqdm to create a progress bar
        results = list(tqdm(p.imap(func, fitindex), total=len(fitindex)))
    
    # Use list comprehension to create 'values' list
    values = [results[i] for i in fitindex]
    
    # Save results in format that is suitable for Machine Learning
    Result = np.column_stack([values, np.asarray(structure_catalogue)])
    np.savetxt(SaveName, Result)
    return Result

def calculate_atom_contribution_value(result, save_results):
    """Calculate atom contribution value list from the result array"""
    
    # Define AtomContributionValues vector
    atom_contribution_values = result[:,0]
    
    # Normalise the AtomContributionValues
    amin, amax = np.min(atom_contribution_values), np.max(atom_contribution_values)
    atom_contribution_values = (atom_contribution_values - amin) / (amax - amin)

    # Define colormap of viridis.reverse
    vmin = np.percentile(atom_contribution_values, 10)
    vmax = np.percentile(atom_contribution_values, 90)
    norm = mpl.colors.Normalize(vmin=vmin, vmax=vmax)
    cmap = mpl.cm.cividis_r
    m = mpl.cm.ScalarMappable(norm=norm, cmap=cmap)
    
    # Save results to file
    with open(f"{save_results}AtomContributionValues_MotEx.txt", "w") as f:
        f.write("\nAtom contribution are calculated to: \n")
        for i, value in enumerate(atom_contribution_values):
            color_code = mpl.colors.rgb2hex(m.to_rgba(value))
            f.write(f"Atom # {i+1}:  {value}  Colorcode:  {color_code}\n")
    
    return m, atom_contribution_values

def Make_CrystalMakerFile(elements, xyz, AtomContributionValues, m, saveResults, threshold):
    # Read bonds and colors of all atoms
    bonding = []
    with open("utils/Bonding.txt", 'r') as fi:
        for line in fi.readlines():
            sep_line=line.strip('{}\n\r ').split()
            bonding.append(sep_line)
    bonding = np.array(bonding)
    
    # Output a crystalmaker file to visualize the results
    with open(f"{saveResults}_CrystalMaker.cmtx", 'w') as CrystalMaker:
        CrystalMaker.write("MOLE  CrystalMaker molecule format\n")
        CrystalMaker.write("TITL  Molecule\n\n")
        CrystalMaker.write("! Model type\n")
        CrystalMaker.write("MODL  1\n\n")
        CrystalMaker.write("! Depth fading settings\n")
        CrystalMaker.write("DCUE  1.000000 0.212899 0.704686\n\n")
        CrystalMaker.write("! Colour definitions:\n")
        CrystalMaker.write("TYPE\n")

        # Assign colors to all the atoms
        for iter, element in enumerate(elements):
            bonding_index = np.where(bonding == element)[0][0]
            if iter < NumMo:
                rgba = m.to_rgba(AtomContributionValues[iter])[:-1]
                rgb = " ".join(map(str, rgba[:3]))
            else:
                rgb = " ".join(map(str, [int(float(bonding[bonding_index, i])*255) for i in range(2, 5)]))
            CrystalMaker.write(f"{element}{iter+1} {bonding[bonding_index, 1]} {rgb}\n")

        CrystalMaker.write("\n! Atoms list\n! Bond Specifications\n")

        # Assign bonds between the atoms
        for iter, element in enumerate(elements):
            if iter < NumMo:
                NI_elements = np.delete(np.unique(elements), np.where(np.unique(elements) == element)[0])
                for NI_element in NI_elements:
                    CrystalMaker.write(f"BMAX {element} {NI_element}  {threshold}\n")

        CrystalMaker.write("\n! Atoms list\nATOM\n")

        # Assign coordinates to the atoms
        for iter, element in enumerate(elements):
            CrystalMaker.write(f"{element} {element}{iter+1} {' '.join(map(str, xyz[iter]))}\n")

    return None

def Make_VESTAFile(elements, xyz, AtomContributionValues, m, saveResults):
    # Read bonds and colors of all atoms
    bonding = []
    with open("utils/Bonding.txt", 'r') as fi:
        for line in fi.readlines():
            sep_line=line.strip('{}\n\r ').split()
            bonding.append(sep_line)
    bonding = np.array(bonding)
    
    # Output a VESTA file to visualize the results
    with open(f"{saveResults}_VESTA.vesta", 'w') as VESTA:
        VESTA.write("VESTA  VESTA format\n")
        VESTA.write("TITL  Molecule\n\n")
        VESTA.write("! Model type\n")
        VESTA.write("MODL  1\n\n")
        VESTA.write("! Colour definitions:\n")
        VESTA.write("TYPE\n")

        # Assign colors to all the atoms
        for iter, element in enumerate(elements):
            bonding_index = np.where(bonding == element)[0][0]
            rgba = m.to_rgba(AtomContributionValues[iter])[:-1]
            rgb = " ".join(map(str, rgba[:3]))
            VESTA.write(f"{element}{iter+1} {bonding[bonding_index, 1]} {rgb}\n")

        VESTA.write("\n! Atoms list\nATOM\n")

        # Assign coordinates to the atoms
        for iter, element in enumerate(elements):
            VESTA.write(f"{element} {element}{iter+1} {' '.join(map(str, xyz[iter]))}\n")

    return None

# Define variables

In [101]:
starting_model = "Structure_Models/OLD_Paper/109725.xyz" # Name of the starting model file
StemName = "Experimental_Data/DanMAX_AlphaKeggin" # Upload PDF(s) from local computer
threshold = 2.6 # Longest bond distance between the metal and oxygen atom
allowed_atoms = ["W"] # The atoms that should be permuted, can be multiple atoms
atom_ph, Qmin, Qmax, Qdamp, rmin, rmax = "W", 0.7, 20, 0.05, 1.6, 10
#atom_ph, Qmin, Qmax, Qdamp, rmin, rmax = "Al", 0.7, 25, 0.04, 1, 10
#atom_ph, Qmin, Qmax, Qdamp, rmin, rmax = "C", 0.5, 24, 0.02, 1, 15
#atom_ph, Qmin, Qmax, Qdamp, rmin, rmax = "Bi", 0.7, 15.5, 0.04, 2, 15
#atom_ph, Qmin, Qmax, Qdamp, rmin, rmax = "Ce", 0.7, 24, 0.04, 1, 20


# Run

In [103]:
saveResults = "./"
start_time = time.time()

# Step 1: Make the structure catalogue
NumMo = format_XYZ(starting_model, allowed_atoms) # Format the starting model and calculate then number of atoms that should be permuted in the starting model
structure_catalogue = structure_catalogue_maker(Number_of_atoms=NumMo)
print ("Catalogue maker: ", time.time() - start_time)

# Step 2: Fit all of the structures from the catalogue of structure motifs to the dataset
### First define the experimental data path and the path you want the structure catalogue with fits to be saved
Experimental_Data = StemName+".gr" # Name of the experimental file
saveFits = StemName+".txt" # Name of the saved fits file

# Load data and start model
Exp_r, Exp_Gr, elements, xyz = Load(Experimental_Data, starting_model)
print ("Data loader: ", time.time() - start_time)

### Produce organized structure catalogue with Rwp values
Result = fitting_multiprocess(structure_catalogue, xyz, atom_ph, Qmin, Qmax, Qdamp, rmin, rmax, SaveName=saveFits, cores=None)
Result[:-1,0] -= Result[-1,0]
print ("Getting results: ", time.time() - start_time)

# Step 4: Calculate Atom Contribution values
m, AtomContributionValues = calculate_atomContributionValue(Result, saveResults)
print ("Calculating atom contribution values: ", time.time() - start_time)

Make_CrystalMakerFile(elements, xyz, AtomContributionValues, m, StemName, threshold)
Make_VestaFile(elements, xyz, AtomContributionValues, m, StemName)
print ("Output a CrystalMaker file: ", time.time() - start_time)

print (time.time() - start_time, " s")


Catalogue maker:  0.002424955368041992
Data loader:  1.5567049980163574


100%|██████████| 25/25 [00:03<00:00,  6.60it/s]

Getting results:  5.581590890884399
Calculating atom contribution values:  5.590118169784546
Output a CrystalMaker file:  5.629946947097778
5.630316972732544  s
